In [94]:
import gymnasium as gym
import numpy as np

In [2]:
!pip install gymnasium

  Obtaining dependency information for gymnasium from https://files.pythonhosted.org/packages/a8/4d/3cbfd81ed84db450dbe73a89afcd8bc405273918415649ac6683356afe92/gymnasium-0.29.1-py3-none-any.whl.metadata
  Obtaining dependency information for farama-notifications>=0.0.1 from https://files.pythonhosted.org/packages/05/2c/ffc08c54c05cdce6fbed2aeebc46348dbe180c6d2c541c7af7ba0aa5f5f8/Farama_Notifications-0.0.4-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/953.9 kB ? eta -:--:--
   ---------------------------------------- 10.2/953.9 kB ? eta -:--:--
   - ------------------------------------- 41.0/953.9 kB 667.8 kB/s eta 0:00:02
   --- ------------------------------------ 92.2/953.9 kB 1.1 MB/s eta 0:00:01
   ------ --------------------------------- 163.8/953.9 kB 1.2 MB/s eta 0:00:01
   --------- ------------------------------ 225.3/953.9 kB 1.1 MB/s eta 0:00:01
   ------------ --------------------------- 286.7/953.9 kB 1.2 MB/s eta 0:00:01
   -------------- ----

In [95]:
env = gym.make("CartPole-v1")

In [96]:
state = env.reset()[0]
for i in range(4):
    state[i] = np.digitize(state[i],state_dimensions[i])
state

array([5., 5., 5., 5.], dtype=float32)

In [32]:
def random_play():
    for episode in range(10):
        env.reset()
        avg_reward = 0
        n=0
        done = False
        while not done:
            n+=1
            env.render()
            
            action = env.action_space.sample()
            
            state,reward,terminated,truncated,info = env.step(action)
            avg_reward += (reward)
            
            done = terminated or truncated
            
        print("average reward is: ")
        print(avg_reward)
    

In [97]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [129]:
env.action_space.n

2

In [154]:
dimension = 10
state_p = np.linspace(-4.8,+4.8,dimension)
state_pv = np.linspace(-4,+4,dimension)
state_a = np.linspace(-0.2095,+0.2095,dimension)
state_av = np.linspace(-4,+4,dimension)

state_dimensions = [state_p,state_pv,state_a,state_av]
rewards_per_episode = []
q_table = np.zeros((dimension+1,dimension+1,dimension+1,dimension+1,2))
episodes = 100000
e_dr = 2/episodes
gamma = 0.99
alpha = 0.1
e = 0.1


for episode in range(episodes):
    rewards = 0
    state = list(env.reset()[0])
    done = False
    
    #digitizing
    state_p = np.digitize(state[0], state_dimensions[0])
    state_v = np.digitize(state[1], state_dimensions[1])
    state_a = np.digitize(state[2], state_dimensions[2])
    state_av = np.digitize(state[3], state_dimensions[3])

    while not done:
        
        env.render()

        #chosing action
        
        if np.random.rand() < e:
            # Choose random action  (0=go left, 1=go right)
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[state_p, state_v, state_a, state_av, :])

        new_state,reward,terminated,_,_ = env.step(action)
        new_state_p = np.digitize(new_state[0], state_dimensions[0])
        new_state_v = np.digitize(new_state[1], state_dimensions[1])
        new_state_a = np.digitize(new_state[2], state_dimensions[2])
        new_state_av= np.digitize(new_state[3], state_dimensions[3])

        q_table[state_p, state_v, state_a, state_av, action] = q_table[state_p, state_v, state_a, state_av, action] + alpha * (
            reward + gamma*np.max(q_table[new_state_p, new_state_v, new_state_a, new_state_av,:]) - q_table[state_p, state_v, state_a, state_av, action]
        )
        state = new_state
        state_p = new_state_p
        state_v = new_state_v
        state_a = new_state_a
        state_av= new_state_av
        rewards += reward

        done = terminated or truncated
    e = max((e-e_dr),0)    
    rewards_per_episode.append(rewards)
    mean_rewards = np.mean(rewards_per_episode[len(rewards_per_episode)-100:])
        
    if truncated:
        print("truncated")

    
    if episode%1000 == 0:
        print(f"{mean_rewards}, {episode},{e}")

9.0, 0,0.09998
16.73, 1000,0.07997999999999388
30.27, 2000,0.05997999999998863
78.94, 3000,0.03997999999998945
82.82, 4000,0.019979999999990262
91.0, 5000,0
86.65, 6000,0
90.53, 7000,0
92.84, 8000,0
91.69, 9000,0
91.23, 10000,0
93.35, 11000,0
94.96, 12000,0
96.67, 13000,0
99.12, 14000,0
94.99, 15000,0
98.13, 16000,0
97.43, 17000,0
94.29, 18000,0
98.16, 19000,0
96.66, 20000,0
97.68, 21000,0
98.77, 22000,0
95.99, 23000,0
102.02, 24000,0
96.17, 25000,0
95.91, 26000,0
95.03, 27000,0
97.72, 28000,0
99.95, 29000,0
94.79, 30000,0
96.8, 31000,0
95.07, 32000,0
99.46, 33000,0
99.15, 34000,0
95.36, 35000,0
98.22, 36000,0
98.74, 37000,0
98.06, 38000,0
99.85, 39000,0
96.4, 40000,0
98.45, 41000,0
95.69, 42000,0
96.5, 43000,0
99.08, 44000,0
97.18, 45000,0
95.65, 46000,0
95.64, 47000,0
96.11, 48000,0
98.26, 49000,0
95.67, 50000,0
95.41, 51000,0
94.4, 52000,0
93.81, 53000,0
98.58, 54000,0
93.82, 55000,0
98.94, 56000,0
96.59, 57000,0
96.79, 58000,0
95.1, 59000,0
98.73, 60000,0
97.16, 61000,0
94.86, 6200

In [121]:
print(sum(q_table!=0))

[[[[0 0]
   [0 0]
   [0 0]
   ...
   [0 0]
   [0 0]
   [0 0]]

  [[0 0]
   [0 0]
   [0 0]
   ...
   [0 0]
   [0 0]
   [0 0]]

  [[0 0]
   [0 0]
   [0 0]
   ...
   [0 0]
   [0 0]
   [0 0]]

  ...

  [[0 0]
   [0 0]
   [0 0]
   ...
   [0 0]
   [0 0]
   [0 0]]

  [[0 0]
   [0 0]
   [0 0]
   ...
   [0 0]
   [0 0]
   [0 0]]

  [[0 0]
   [0 0]
   [0 0]
   ...
   [0 0]
   [0 0]
   [0 0]]]


 [[[0 0]
   [0 0]
   [0 0]
   ...
   [0 0]
   [0 0]
   [0 0]]

  [[0 0]
   [0 0]
   [0 0]
   ...
   [0 0]
   [0 0]
   [0 0]]

  [[0 0]
   [0 0]
   [0 0]
   ...
   [0 0]
   [0 0]
   [0 0]]

  ...

  [[0 0]
   [0 0]
   [0 0]
   ...
   [0 0]
   [0 0]
   [0 0]]

  [[0 0]
   [0 0]
   [0 0]
   ...
   [0 0]
   [0 0]
   [0 0]]

  [[0 0]
   [0 0]
   [0 0]
   ...
   [0 0]
   [0 0]
   [0 0]]]


 [[[0 0]
   [0 0]
   [0 0]
   ...
   [0 0]
   [0 0]
   [0 0]]

  [[0 0]
   [0 0]
   [0 0]
   ...
   [0 0]
   [0 0]
   [0 0]]

  [[0 0]
   [0 0]
   [0 0]
   ...
   [0 0]
   [0 0]
   [0 0]]

  ...

  [[0 0]
   [0 0]
   [0 0]
   

In [ ]:
print(q_table[state[0],state[1],state[2],state[3],action])

Episode: 0 11.0  Epsilon: 1.00  Mean Rewards 11.0
Episode: 100 16.0  Epsilon: 1.00  Mean Rewards 22.7
Episode: 200 16.0  Epsilon: 1.00  Mean Rewards 21.6
Episode: 300 14.0  Epsilon: 1.00  Mean Rewards 23.7
Episode: 400 17.0  Epsilon: 1.00  Mean Rewards 21.2
Episode: 500 29.0  Epsilon: 1.00  Mean Rewards 23.3
Episode: 600 23.0  Epsilon: 0.99  Mean Rewards 22.6
Episode: 700 19.0  Epsilon: 0.99  Mean Rewards 23.1
Episode: 800 18.0  Epsilon: 0.99  Mean Rewards 21.4
Episode: 900 18.0  Epsilon: 0.99  Mean Rewards 24.7
Episode: 1000 24.0  Epsilon: 0.99  Mean Rewards 22.4
Episode: 1100 21.0  Epsilon: 0.99  Mean Rewards 23.6
Episode: 1200 15.0  Epsilon: 0.99  Mean Rewards 21.9
Episode: 1300 13.0  Epsilon: 0.99  Mean Rewards 22.9
Episode: 1400 41.0  Epsilon: 0.99  Mean Rewards 20.5
Episode: 1500 16.0  Epsilon: 0.99  Mean Rewards 22.1
Episode: 1600 20.0  Epsilon: 0.98  Mean Rewards 24.2
Episode: 1700 14.0  Epsilon: 0.98  Mean Rewards 23.8
Episode: 1800 68.0  Epsilon: 0.98  Mean Rewards 22.9
Episo

Episode: 15400 22.0  Epsilon: 0.85  Mean Rewards 29.4
Episode: 15500 11.0  Epsilon: 0.85  Mean Rewards 30.8
Episode: 15600 58.0  Epsilon: 0.84  Mean Rewards 29.0
Episode: 15700 28.0  Epsilon: 0.84  Mean Rewards 28.5
Episode: 15800 19.0  Epsilon: 0.84  Mean Rewards 29.9
Episode: 15900 35.0  Epsilon: 0.84  Mean Rewards 32.6
Episode: 16000 28.0  Epsilon: 0.84  Mean Rewards 33.3
Episode: 16100 30.0  Epsilon: 0.84  Mean Rewards 30.9
Episode: 16200 53.0  Epsilon: 0.84  Mean Rewards 33.5
Episode: 16300 13.0  Epsilon: 0.84  Mean Rewards 29.9
Episode: 16400 23.0  Epsilon: 0.84  Mean Rewards 34.1
Episode: 16500 55.0  Epsilon: 0.84  Mean Rewards 31.8
Episode: 16600 14.0  Epsilon: 0.83  Mean Rewards 30.9
Episode: 16700 13.0  Epsilon: 0.83  Mean Rewards 33.1
Episode: 16800 45.0  Epsilon: 0.83  Mean Rewards 31.5
Episode: 16900 15.0  Epsilon: 0.83  Mean Rewards 32.7
Episode: 17000 36.0  Epsilon: 0.83  Mean Rewards 29.0
Episode: 17100 14.0  Epsilon: 0.83  Mean Rewards 30.1
Episode: 17200 36.0  Epsilon

Episode: 30700 45.0  Epsilon: 0.69  Mean Rewards 45.0
Episode: 30800 18.0  Epsilon: 0.69  Mean Rewards 44.5
Episode: 30900 32.0  Epsilon: 0.69  Mean Rewards 45.0
Episode: 31000 35.0  Epsilon: 0.69  Mean Rewards 45.1
Episode: 31100 13.0  Epsilon: 0.69  Mean Rewards 41.1
Episode: 31200 19.0  Epsilon: 0.69  Mean Rewards 49.6
Episode: 31300 33.0  Epsilon: 0.69  Mean Rewards 44.9
Episode: 31400 60.0  Epsilon: 0.69  Mean Rewards 47.9
Episode: 31500 36.0  Epsilon: 0.69  Mean Rewards 49.5
Episode: 31600 84.0  Epsilon: 0.68  Mean Rewards 45.1
Episode: 31700 76.0  Epsilon: 0.68  Mean Rewards 49.5
Episode: 31800 45.0  Epsilon: 0.68  Mean Rewards 46.7
Episode: 31900 30.0  Epsilon: 0.68  Mean Rewards 52.6
Episode: 32000 96.0  Epsilon: 0.68  Mean Rewards 45.1
Episode: 32100 38.0  Epsilon: 0.68  Mean Rewards 46.7
Episode: 32200 12.0  Epsilon: 0.68  Mean Rewards 43.3
Episode: 32300 31.0  Epsilon: 0.68  Mean Rewards 46.4
Episode: 32400 23.0  Epsilon: 0.68  Mean Rewards 48.7
Episode: 32500 15.0  Epsilon

Episode: 45900 74.0  Epsilon: 0.54  Mean Rewards 84.1
Episode: 46000 179.0  Epsilon: 0.54  Mean Rewards 80.0
Episode: 46100 177.0  Epsilon: 0.54  Mean Rewards 84.5
Episode: 46200 10.0  Epsilon: 0.54  Mean Rewards 74.6
Episode: 46300 59.0  Epsilon: 0.54  Mean Rewards 77.7
Episode: 46400 102.0  Epsilon: 0.54  Mean Rewards 81.3
Episode: 46500 11.0  Epsilon: 0.54  Mean Rewards 67.0
Episode: 46600 98.0  Epsilon: 0.53  Mean Rewards 80.9
Episode: 46700 153.0  Epsilon: 0.53  Mean Rewards 76.0
Episode: 46800 147.0  Epsilon: 0.53  Mean Rewards 82.9
Episode: 46900 129.0  Epsilon: 0.53  Mean Rewards 88.7
Episode: 47000 88.0  Epsilon: 0.53  Mean Rewards 81.6
Episode: 47100 50.0  Epsilon: 0.53  Mean Rewards 83.6
Episode: 47200 128.0  Epsilon: 0.53  Mean Rewards 91.0
Episode: 47300 65.0  Epsilon: 0.53  Mean Rewards 78.9
Episode: 47400 135.0  Epsilon: 0.53  Mean Rewards 74.4
Episode: 47500 115.0  Epsilon: 0.53  Mean Rewards 91.1
Episode: 47600 113.0  Epsilon: 0.52  Mean Rewards 81.7
Episode: 47700 65.

Episode: 60800 75.0  Epsilon: 0.39  Mean Rewards 143.9
Episode: 60900 265.0  Epsilon: 0.39  Mean Rewards 145.6
Episode: 61000 326.0  Epsilon: 0.39  Mean Rewards 141.7
Episode: 61100 253.0  Epsilon: 0.39  Mean Rewards 168.7
Episode: 61200 90.0  Epsilon: 0.39  Mean Rewards 146.5
Episode: 61300 98.0  Epsilon: 0.39  Mean Rewards 139.3
Episode: 61400 13.0  Epsilon: 0.39  Mean Rewards 159.3
Episode: 61500 130.0  Epsilon: 0.39  Mean Rewards 143.8
Episode: 61600 18.0  Epsilon: 0.38  Mean Rewards 148.8
Episode: 61700 244.0  Epsilon: 0.38  Mean Rewards 154.4
Episode: 61800 164.0  Epsilon: 0.38  Mean Rewards 141.6
Episode: 61900 357.0  Epsilon: 0.38  Mean Rewards 154.7
Episode: 62000 144.0  Epsilon: 0.38  Mean Rewards 156.5
Episode: 62100 195.0  Epsilon: 0.38  Mean Rewards 160.9
Episode: 62200 105.0  Epsilon: 0.38  Mean Rewards 165.3
Episode: 62300 305.0  Epsilon: 0.38  Mean Rewards 143.0
Episode: 62400 128.0  Epsilon: 0.38  Mean Rewards 155.2
Episode: 62500 43.0  Epsilon: 0.38  Mean Rewards 159.

Episode: 75500 223.0  Epsilon: 0.25  Mean Rewards 319.7
Episode: 75600 83.0  Epsilon: 0.24  Mean Rewards 282.0
Episode: 75700 395.0  Epsilon: 0.24  Mean Rewards 306.9
Episode: 75800 317.0  Epsilon: 0.24  Mean Rewards 295.7
Episode: 75900 435.0  Epsilon: 0.24  Mean Rewards 255.1
Episode: 76000 449.0  Epsilon: 0.24  Mean Rewards 315.4
Episode: 76100 492.0  Epsilon: 0.24  Mean Rewards 327.2
Episode: 76200 269.0  Epsilon: 0.24  Mean Rewards 322.8
Episode: 76300 239.0  Epsilon: 0.24  Mean Rewards 331.1
Episode: 76400 273.0  Epsilon: 0.24  Mean Rewards 292.9
Episode: 76500 308.0  Epsilon: 0.24  Mean Rewards 322.8
Episode: 76600 155.0  Epsilon: 0.23  Mean Rewards 343.6
Episode: 76700 750.0  Epsilon: 0.23  Mean Rewards 339.1
Episode: 76800 915.0  Epsilon: 0.23  Mean Rewards 328.6
Episode: 76900 156.0  Epsilon: 0.23  Mean Rewards 309.8
Episode: 77000 23.0  Epsilon: 0.23  Mean Rewards 331.1
Episode: 77100 214.0  Epsilon: 0.23  Mean Rewards 346.8
Episode: 77200 296.0  Epsilon: 0.23  Mean Rewards 

KeyboardInterrupt: 